# Imports

In [45]:
# !pip install pypdf spacy pandas wordcloud
# !python -m spacy download en_core_web_sm
# !python -m spacy download en_core_web_lg
import re
from pathlib import Path
# import PyPDF2 as pdf2
import pypdf as pdf2
import numpy as np
import pandas as pd
import nltk
import spacy
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
# nltk.download(['stopwords', 'wordnet'])
stop_words = stopwords.words('english')

from wordcloud import WordCloud

Traceback (most recent call last):
  File "/Users/PT46FV/Developer/Learning/master-nlp/fnlp/venv/lib/python3.9/site-packages/urllib3/util/url.py", line 425, in parse_url
    host, port = _HOST_PORT_RE.match(host_port).groups()  # type: ignore[union-attr]
AttributeError: 'NoneType' object has no attribute 'groups'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/PT46FV/Developer/Learning/master-nlp/fnlp/venv/lib/python3.9/site-packages/requests/adapters.py", line 454, in send
    conn = self.get_connection(request.url, proxies)
  File "/Users/PT46FV/Developer/Learning/master-nlp/fnlp/venv/lib/python3.9/site-packages/requests/adapters.py", line 343, in get_connection
    proxy = prepend_scheme_if_needed(proxy, "http")
  File "/Users/PT46FV/Developer/Learning/master-nlp/fnlp/venv/lib/python3.9/site-packages/requests/utils.py", line 993, in prepend_scheme_if_needed
    parsed = parse_url(url)
  File "/Users/PT46FV/Deve

# Configs


In [24]:
ignore_pages = {
    "14 REMARKS ON COLOUR": [0, 35],
    "13 ON CERTAINTY": [0, 1, 51],
    "12 LAST WRITINGS ON THE PHILOSOPHY OF PSYCHOLOGY 2": [0, 1, 2, 3, 44, 45],
    "11 LAST WRITINGS ON THE PHILOSOPHY OF PSYCHOLOGY": [0, 1, 73, 74, 75, 76],
    "10 REMARKS ON THE PHILOSOPHY OF PSYCHOLOGY 2": [0, 1, 68, 69, 70, 71, 72, 73, 74, 75],
    "09 REMARKS ON THE PHILOSOPHY OF PSYCHOLOGY 1": [0, 1, 2, 110, 111],
    "08 PHILOSOPHICAL INVESTIGATIONS": [0, 1, 2, 3, 4],
    "07 ZETTEL": [0, 1, 2],
    "06 Culture and Value": [0, 1, 2, 3, 4, 5, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107],
    "05 REMARKS ON THE FOUNDATIONS OF MATHEMATICS": [0, 1, 2, 3, 4, 5, 172, 173, 174],
    "04 THE BLUE AND THE BROWN BOOKS": [0, 1, 2, 3, 4, 5, 6, 7, 106],
    "03 PHILOSOPHICAL GRAMMAR": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 227, 228, 229, 230, 231, 232, 233],
    "02 PHILOSOPHICAL REMARKS": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 163, 164, 165, 166, 167, 168, 168, 170, 171],
    "01 Notebooks": [0, 1, 93, 94, 95, 96, 97, 98, 99],
    "(Routledge Classics) Ludwig Wittgenstein - Tractatus Logico-Philosophicus-Routledge (2001)": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134],
    "Wittgenstein - Notebooks 1914-1916 ed 1961": []
}


# Preprocessing
## Extract PDFs to txt file

In [25]:
def _clean_text(text):
    text = re.sub(r"Page( Break)? \d{1,3}" , " ", text)
    text = re.sub(r"Page( Break)? [ivm]{1,3}" , " ", text) 
    # remove capital letter words
    text = re.sub(r"/(\w*[A-Z]+\w*)\w+/", "", text)
    text = re.sub("\d{1,5}.", "", text)
    text = re.sub(r"\s+" , " ", text)
    text = re.sub(r"†", "", text)
    # ( Eds.\)
    text = re.sub(r"\( Eds.\)", "", text)
    # [Cf. Z ]
    text = re.sub(r"\[a-zA-Z\w*\]", "", text)
    # text = re.sub(r"\s+" , " ", text)
    
    return text

def extract_pdf(pdf_file_path, name):
    with open (pdf_file_path, "rb") as f:
        pdf = pdf2.PdfReader(f)
        full_text = ""
        for index, page in enumerate(pdf.pages):
            # skip first page and ignored pages
            if index not in [0] and index not in ignore_pages[name]:
                full_text += _clean_text(page.extract_text() + '\n')
        return full_text
    
def save_file(data: str, file_name: str):
    Path("data").mkdir(parents=True, exist_ok=True)
    file = Path("data") / file_name
    file.write_text(data)

        

In [26]:

for pdf_file in Path('data/raw').glob('*.pdf'):
    file_name = pdf_file.name.split('.')[0] 
    print(file_name)
    # text = extract_pdf(pdf_file, file_name)
    # save_file(text, file_name + '.txt')

07 ZETTEL
14 REMARKS ON COLOUR
09 REMARKS ON THE PHILOSOPHY OF PSYCHOLOGY 1
04 THE BLUE AND THE BROWN BOOKS
02 PHILOSOPHICAL REMARKS
(Routledge Classics) Ludwig Wittgenstein - Tractatus Logico-Philosophicus-Routledge (2001)
10 REMARKS ON THE PHILOSOPHY OF PSYCHOLOGY 2
05 REMARKS ON THE FOUNDATIONS OF MATHEMATICS
08 PHILOSOPHICAL INVESTIGATIONS
01 Notebooks
06 Culture and Value
11 LAST WRITINGS ON THE PHILOSOPHY OF PSYCHOLOGY
12 LAST WRITINGS ON THE PHILOSOPHY OF PSYCHOLOGY 2
Wittgenstein - Notebooks 1914-1916 ed 1961
03 PHILOSOPHICAL GRAMMAR
13 ON CERTAINTY


## Read into df applying normalization:
### Splitting into sentences

In [27]:
def preprocess(text):
    if isinstance(text, str):
        # remove multiple whitespaces
        text = re.sub(r"\s+" , " ", text)
        # remove same repeating characters
        text = re.sub(r"(\w)\1{2,}", r"\1", text)
        # keep comments with at least 5 words
        return re.sub(r'[^a-zA-Z\s]', '', text).lower()

In [55]:
df = pd.DataFrame(columns=['sentence', 'label_book'])
pdf_map = {}
# change to a different larger model
nlp = spacy.load("en_core_web_lg")
stop_words = nlp.Defaults.stop_words
(Path("data/wordcloud").mkdir(parents=True, exist_ok=True))

def generate_df(generate: bool = False):
    if generate:
        for index, pdf_file in enumerate(Path('data').glob('*.txt')):
            file_name = pdf_file.name.split('.')[0]
            pdf_map[index] = file_name
            with open(pdf_file) as f:
                lines = f.readlines()
                for line in lines:
                    sentences_preprocessed = []
                    sentences_lemmatized = []
                    # sentence tokenizer
                    sentences = sent_tokenize(line)
                    for sentence in sentences:
                        sentence = preprocess(sentence) 
                        words = [word for word in sentence.split() if word.lower() not in stop_words and len(word) > 2]
                        sentence_clean = ' '.join(words)
                        # add only sentences with at least 3 words
                        if len(sentence_clean.split(' ')) <= 2:
                            continue
                        # start preprocess
                        sentences_preprocessed.append(sentence_clean)

                    for doc in nlp.pipe(sentences_preprocessed, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "pos_tagger", "ner"]):
                        # Do something with the doc here
                        sentence_lemmatized = " ".join([token.lemma_ for token in doc])
                        sentences_lemmatized.append(sentence_lemmatized)
                    # wordclouds save
                    try:
                        word_cloud = WordCloud(
                                collocations=False,
                                background_color='black',
                                width=2048, height=1080,
                                ).generate(" ".join(sentences_lemmatized))
                        word_cloud.to_file(Path("data/wordcloud") / f"{file_name}_wc.png")
                    except Exception as e:
                        print(e)
                    labels = np.full(len(sentences_lemmatized), file_name)
                    temp_df = pd.DataFrame({'sentence': sentences_lemmatized, 'label_book': index})
                    df = pd.concat([df, temp_df])
                    df.to_csv((Path("data")) / "dataframe.csv")
    else:
        df = pd.read_csv((Path("data")) / "dataframe.csv")
        
    return df


df = generate_df()

In [56]:
df

,Unnamed: 0,sentence,label_book
0,0,world totality facts things,0
1,1,world determined facts facts,0
2,2,totality facts determines case lso casethe fac...,0
3,3,world divides facts,0
4,4,ach item case case hing remains samewhat casea...,0
...,...,...,...
38307,1075,melding bothas tempted,15
38308,1076,asks way difficult answer,15
38309,1077,arrow end remark shows connected following remark,15
38310,1078,sentence obviously incomplete,15
